# Demo de algoritmos para almacenar passwords

In [ ]:
password = b'cbda'

In [ ]:
ALPHABET = 'abcd'

## Probando las funciones hash clásicas: MD5, SHA1 y SHA256

In [ ]:
from Crypto.Hash import MD5, SHA, SHA256

In [ ]:
md5 = MD5.new()
md5.update(password)
print(md5.hexdigest())

In [ ]:
sha1 = SHA.new()
sha1.update(password)
print(sha1.hexdigest())

In [ ]:
sha256 = SHA256.new()
sha256.update(password)
print(sha256.hexdigest())

Ahora, una función que me permita generar todas las passwords posibles (del largo del alfabeto) para poder hacer fuerza bruta, y una segunda que me permita hacer la fuerza bruta para cada algoritmo:

In [ ]:
import itertools

def get_bruteforce_string_list():
    a = itertools.product(ALPHABET, repeat = len(ALPHABET))
    while a:
        yield "".join(a.next())

In [ ]:
def bruteforce_hash(hash_object, password_hash):
    for attempt in get_bruteforce_string_list():
        if hash_object.new(attempt).hexdigest() == password_hash.hexdigest():
            print('PASSWORD FOUND: %s' % attempt)
            break

#### Fuerza bruta para texto plano

In [ ]:
def bruteforce_plaintext():
    for attempt in get_bruteforce_string_list():
        if attempt == password:
            print('PASSWORD FOUND: %s' % attempt)
            break
timeit_plaintext = %%timeit -n1 -r1 -q -o bruteforce_plaintext()

#### Fuerza bruta para MD5

In [ ]:
timeit_md5 = %%timeit -n1 -r1 -q -o bruteforce_hash(MD5, md5)

#### Fuerza bruta para SHA1

In [ ]:
timeit_sha1 = %%timeit -n1 -r1 -q -o bruteforce_hash(SHA, sha1)

#### Fuerza bruta para SHA256

In [ ]:
timeit_sha256 = %%timeit -n1 -r1 -q -o bruteforce_hash(SHA256, sha256)

## Ahora con una función KDF: BCrypt

In [ ]:
import bcrypt

Generación de una sal. A la función `gensalt` se le puede dar un parámetro de costo. Por defecto es 12. Ver <https://en.wikipedia.org/wiki/Bcrypt>

In [ ]:
salt = bcrypt.gensalt()
print(salt)

Generación del KDF o "digest" respectivo:

In [ ]:
kdf = bcrypt.hashpw(password, salt)
print(kdf)

#### Fuerza bruta para BCrypt

In [ ]:
def bruteforce_bcrypt():
    for attempt in get_bruteforce_string_list():
        if bcrypt.hashpw(attempt, kdf) == kdf:
            print('PASSWORD FOUND: %s' % attempt)
            break
timeit_kdf = %%timeit -n1 -r1 -q -o bruteforce_bcrypt()

## Resultados

In [ ]:
from ipy_table import make_table, apply_theme
results_data = [
    ['Tipo', 'Tiempo (s)'],
    ['Texto plano', timeit_plaintext.best],
    ['MD5', timeit_md5.best],
    ['SHA1', timeit_sha1.best],
    ['SHA256', timeit_sha256.best],
    ['KDF (BCrypt)', timeit_kdf.best],
]
make_table(results_data)
apply_theme('basic')